In [35]:
import pandas as pd 
import numpy as np 

In [231]:
df = pd.read_csv("ASTUTE DYNAMIC FUND.csv", index_col=1, parse_dates=True).drop("Unnamed: 0", axis=1)
df.index = pd.to_datetime(df.index, format="%d/%m/%Y")
df = df.loc[:"2024-03-31", :].sort_index()
# index = pd.date_range("2018-01-01", "2024-03-31", freq='D')
# df = pd.DataFrame(np.random.random(len(index)), index=index, columns=['Last'])
df

,NAV Price
NAV Date,
2006-09-20,0.2496
2006-09-21,0.2507
2006-09-22,0.2517
2006-09-25,0.2512
2006-09-26,0.2516
...,...
2024-03-22,0.2198
2024-03-25,0.2225
2024-03-26,0.2244


In [210]:
class Financial_metrics_calculation:

    YTD_RISK_FREE_RATE =0.0067
    ONE_Y_RISK_FREE_RATE = 0.028
    THREE_Y_RISK_FREE_RATE = 0.0734
    FIVE_Y_RISK_FREE_RATE = 0.1276
    TEN_Y_RISK_FREE_RATE = 0.3228

    RISK_FREE_RATES = pd.DataFrame([YTD_RISK_FREE_RATE, ONE_Y_RISK_FREE_RATE, THREE_Y_RISK_FREE_RATE, FIVE_Y_RISK_FREE_RATE, TEN_Y_RISK_FREE_RATE], index=["YTD Risk Free Rate", "1Y Risk Free Rate", "3Y Risk Free Rate", "5Y Risk Free Rate", "10Y Risk Free Rate"]).T

    def __init__(self, df):
        self.df = df
        self.month_end_data = self.df.resample("M").last()
        self.years = self.df.index.year.unique()

    def compute_returns(self, return_df):
        return (return_df.iloc[-1, :] / return_df.iloc[0, :]) - 1

    def annual_return(self):
        return_df = pd.DataFrame()
        for year in self.years:
            return_df[f"{year} Return"] = self.compute_returns(self.df.loc[str(year), :])
        return return_df

    def fund_returns(self):
        return_df = pd.DataFrame(columns=["YTD Return", "1Y Return", "3Y Return", "5Y Return", "10Y Return"])
        return_df["YTD Return"] = self.compute_returns(self.df.loc["2024", :])
        years = [1, 3, 5, 10]
        for year in years:
            end_date = self.df.index[-1].date()
            start_date = end_date - pd.DateOffset(years=year)
            earliest_date = self.df.index[0].date()        
            if start_date.date() >= earliest_date:
                period_df = self.df.loc[str( start_date ): str(end_date)] 
                return_df[f"{year}Y Return"] = self.compute_returns(period_df)
            else:
                return_df[f"{year}Y Return"] = np.nan
        return return_df

    def compute_std(self, period_df):
        return_df = period_df.pct_change()
        return return_df.std(ddof=1)

    def annual_std(self):
        std_df = pd.DataFrame()
        for year in self.years:
            std_df[f"{year} Std"] = self.compute_std(self.df.loc[str(year), :])
        return std_df

    def fund_std(self):
        std_df = pd.DataFrame(columns=["YTD Std", "1Y Std", "3Y Std", "5Y Std", "10Y Std"])
        std_df["YTD Std"] = self.compute_std(self.df.loc["2024", :])
        years = [1, 3, 5, 10]
        for year in years:
            end_date = self.df.index[-1].date()
            start_date = end_date - pd.DateOffset(years=year)
            earliest_date = self.df.index[0].date()        
            if start_date.date() >= earliest_date:
                period_df = self.df.loc[str( start_date ): str(end_date)]
                std_df[f"{year}Y Std"] = self.compute_std(period_df)
            else:
                std_df[f"{year}Y Std"] = np.nan
        return std_df

    def annualized_return(self, return_df):
        target_return = return_df.iloc[:5, :]
        temp_df = pd.concat([target_return, pd.Series([self.df.index[-1].month/12, 1, 3, 5, 10], index=target_return.index)], axis=1)
        annualized_df = (1 + temp_df.iloc[:, 0]) ** (1/temp_df.iloc[:, -1]) - 1
        return_df.iloc[:5, :] = annualized_df.to_frame()
        return return_df

    def annuazlied_std(self, std_df):
        return std_df * np.sqrt(252)
    
    def fund_sharpe(self):
        fund_returns = self.annualized_return( pd.concat([self.fund_returns(), self.annual_return()], axis=1).T)
        fund_stds = self.annuazlied_std( pd.concat([self.fund_std(), self.annual_std()], axis=1).T )
        fund_risk_free_rates = pd.DataFrame(np.full_like(fund_returns, Financial_metrics_calculation.ONE_Y_RISK_FREE_RATE), index=fund_returns.index)
        sharpe_df = pd.concat([fund_returns.reset_index(drop=True), fund_risk_free_rates.reset_index(drop=True), fund_stds.reset_index(drop=True)], axis=1, ignore_index=True)
        sharpe_df = sharpe_df.rename(columns = dict(zip(sharpe_df.columns, ["Returns", "Risk Free Rate", "Std"])))
        sharpe_df.index = ["YTD Sharpe", "1Y Sharpe", "3Y Sharpe", "5Y Sharpe", "10Y Sharpe"] + [f"{year} Sharpe" for year in self.years]
        sharpe_ratio_df = ((sharpe_df['Returns'] - sharpe_df['Risk Free Rate']) / sharpe_df['Std']).to_frame()
        sharpe_ratio_df.columns = ["Last"]
        return sharpe_ratio_df

    def max_drawdown(self):
        max_nav = self.df.cummax()
        diff_nav_max = self.df - max_nav
        drawdown = diff_nav_max / max_nav
        max_drawdown = drawdown.min()
        # print(max_drawdown)
        return max_drawdown

In [232]:
df.to_clipboard(excel=True)

In [229]:
# x= fund.annualized_return(fund.fund_returns().T)
x = fund.annual_return()
x

,2006 Return,2007 Return,2008 Return,2009 Return,2010 Return,2011 Return,2012 Return,2013 Return,2014 Return,2015 Return,2016 Return,2017 Return,2018 Return,2019 Return,2020 Return,2021 Return,2022 Return,2023 Return,2024 Return
NAV Price,0.106971,-0.051437,-0.334331,0.323596,0.147708,-0.036982,0.034562,0.053711,-0.159646,0.062342,-0.254163,0.110048,-0.272989,0.107641,0.025795,0.033333,-0.060555,0.111245,0.208513


In [233]:
fund.fund_returns()

,YTD Return,1Y Return,3Y Return,5Y Return,10Y Return
NAV Price,0.208513,0.401875,0.206563,0.377764,-0.181387


In [187]:
y = fund.annuazlied_std(fund.fund_std().T)
y

,Last
YTD Std,281.673498
1Y Std,150.736581
3Y Std,6755.318016
5Y Std,5237.939979
10Y Std,NaN


In [189]:
(0.515226 - 0.028) / 5237.939979

9.301862983413158e-05

In [211]:
fund = Financial_metrics_calculation(df)
x= fund.fund_sharpe()
x

,Last
YTD Sharpe,7.979460
1Y Sharpe,3.747869
3Y Sharpe,0.363296
5Y Sharpe,0.298624
10Y Sharpe,-0.404283
2006 Sharpe,1.001646
2007 Sharpe,-0.365370
2008 Sharpe,-2.802161
2009 Sharpe,3.619010
2010 Sharpe,1.527187


In [88]:

df.index[-1].month

3

In [39]:
# date  = df.index[0].date() 
end_date = df.index[-1].date()
start_date = end_date - pd.DateOffset(years=1)
earliest_date = df.index[0].date()
start_date.date() >= earliest_date

True

In [ ]:
date = 
df.loc[]

In [215]:
financial_metrics = Financial_metrics_calculation(df)
# returns = pd.concat([financial_metrics.fund_returns(), financial_metrics.annual_return()], axis=1)
# sharpe_ratios = financial_metrics.fund_sharpe().T
# max_drawdown = financial_metrics.max_drawdown()
# combined_df = pd.concat([returns, sharpe_ratios], axis=1)
combined_df['Max Drawdown'] = financial_metrics.max_drawdown()
combined_df

NameError: name 'combined_df' is not defined

In [174]:
combined_df[['1', '2', '3']] = [1,2,3]
combined_df

,YTD Return,1Y Return,3Y Return,5Y Return,10Y Return,2017 Return,2018 Return,2019 Return,2020 Return,2021 Return,...,2019 Sharpe,2020 Sharpe,2021 Sharpe,2022 Sharpe,2023 Sharpe,2024 Sharpe,Max Drawdown,1,2,3
Last,0.080346,0.107709,-0.044818,-0.022857,NaN,0.029619,-0.124338,0.051482,-0.009645,-0.050033,...,5.240447,-3.889022,-13.87035,-9.833917,-6.828101,12.31985,-0.309904,1,2,3


In [216]:
df = pd.read_csv("Malaysia Fund Universe.csv", index_col=0)
df

,A-Dana Equity,A-DALI Equity,A-Dana Income,A-Dana Balanced,Aberdeen Std Islmc World Eq A,Advantage Glbl Eq Volatility Focus AUD H,Advantage Glbl Eq Volatility Focus MYR H,Advantage Asia Pacific ex Japan Div,Advantage Glb High Income Bond RM-Hdg,Advantage Glb High Income Bond USD,...,United Sustainable Series Glbl Crdt SGDH,United USD Cash USD,United Vietnam Equities AUD Hedged,United Vietnam Equities GBP Hedged,United Vietnam Equities MYR Hedged,United Vietnam Equities RMB Hedged,United Vietnam Equities SGD Hedged,United Vietnam Equities USD,US-Canada Income and Growth,Principal Greater China Equity RMB H
YTD Return,0.0841995318752673,0.08034566339972593,0.014032602560839269,0.06791773711871252,0.11172013541834591,0.059432672353361804,0.054144778987828435,0.0551462591145957,0.0034688293922220392,0.01210946476565411,...,0.0005762360262762112,0.01225110719043676,0.12919999999999998,0.0,0.13677445270134547,0.0,0.0,0.14214214214214205,0.04561179386289882,-0.008327918561836678
1Y Return,0.11812341833755102,0.10770859896700014,0.06457459539217636,0.11044213466767006,0.28776844070961727,0.1433600743054464,0.12525729134612096,0.12034624209833966,0.03459566194174957,0.07341238304140041,...,0.02551381998582558,0.053088477674558154,NaN,NaN,NaN,NaN,NaN,NaN,0.18419042327858604,-0.1873718773572347
3Y Return,-0.0022697532222389905,-0.04481756177555818,0.1394657646678441,0.05146988013687648,0.3110266159695816,0.06431549735737496,0.07447093315572628,-0.01175866177128404,-0.11882309533820135,-0.08658172604394909,...,NaN,0.08438602919905103,NaN,NaN,NaN,NaN,NaN,NaN,0.1353578285042114,-0.4027575771410682
5Y Return,0.32351721277478207,-0.022857251791978173,0.2507541735486427,0.3556210372970021,0.778694867165334,0.2944868000373144,0.3513899724879894,0.2538970745248772,-0.01036898555473098,0.013466329174712133,...,NaN,0.1102411958099263,NaN,NaN,NaN,NaN,NaN,NaN,0.48937672414420574,NaN
10Y Return,NaN,NaN,NaN,NaN,1.2885208907178178,NaN,NaN,0.9398070068677735,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1982 Return,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1983 Return,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1981 Sharpe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1982 Sharpe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [228]:
index = [str(i) + " Return" for i in range(1981, 2025)] + ["YTD Return", "1Y Return", "3Y Return", "5Y Return", "10Y Return"] + \
        [str(i) + " Sharpe" for i in range(1981, 2025)] + ["YTD Sharpe", "1Y Sharpe", "3Y Sharpe", "5Y Sharpe", "10Y Sharpe"] +\
        ['Max Drawdown', "PriceCurrency", "StarRatingM255", "CategoryName"]
df.loc[index,:].T.to_csv("Malaysia Fund Universe (Updated).csv")

In [220]:
for i in df.index:
    print(i)

YTD Return
1Y Return
3Y Return
5Y Return
10Y Return
2017 Return
2018 Return
2019 Return
2020 Return
2021 Return
2022 Return
2023 Return
2024 Return
YTD Sharpe
1Y Sharpe
3Y Sharpe
5Y Sharpe
10Y Sharpe
2017 Sharpe
2018 Sharpe
2019 Sharpe
2020 Sharpe
2021 Sharpe
2022 Sharpe
2023 Sharpe
2024 Sharpe
Max Drawdown
PriceCurrency
StarRatingM255
CategoryName
2013 Return
2014 Return
2015 Return
2016 Return
2013 Sharpe
2014 Sharpe
2015 Sharpe
2016 Sharpe
2012 Return
2012 Sharpe
2007 Return
2008 Return
2009 Return
2010 Return
2011 Return
2007 Sharpe
2008 Sharpe
2009 Sharpe
2010 Sharpe
2011 Sharpe
2001 Return
2002 Return
2003 Return
2004 Return
2005 Return
2006 Return
2001 Sharpe
2002 Sharpe
2003 Sharpe
2004 Sharpe
2005 Sharpe
2006 Sharpe
1993 Return
1994 Return
1995 Return
1996 Return
1997 Return
1998 Return
1999 Return
2000 Return
1993 Sharpe
1994 Sharpe
1995 Sharpe
1996 Sharpe
1997 Sharpe
1998 Sharpe
1999 Sharpe
2000 Sharpe
1986 Return
1987 Return
1988 Return
1989 Return
1990 Return
1991 Return
1